In [1]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import requests
import datetime as dt
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from sklearn.linear_model import LinearRegression

coin = 'ETHUSDT'
key = "https://api.binance.com/api/v3/ticker?symbol=" + coin

app = dash.Dash(__name__, update_title=None)

totalErr = []
average_size = 10
errList = []
priceList = []
predictList = []
dateList = []
rmseList = []

model = LinearRegression()

app.layout = html.Div(
    [
        dcc.Graph(id='price-graph'),
        dcc.Graph(id='rmse-graph'),
        dcc.Interval(id="interval", interval=1 * 300)
    ]
)

@app.callback(
    [Output('price-graph', 'figure'), Output('rmse-graph', 'figure')],
    [Input('interval', 'n_intervals')])
def update_data(n_intervals):
    global totalErr, errList, priceList, predictList, dateList, rmseList
    data = requests.get(key).json()
    price = float(data['lastPrice'])
    closeTime = data['closeTime']
    my_datetime = dt.datetime.fromtimestamp(closeTime / 1000)

    dateList.append(my_datetime)
    priceList.append(price)

    X = np.array(range(len(priceList))).reshape(-1, 1)
    y = np.array(priceList)

    if len(priceList) > 1:
        model.fit(X, y)
        predicted = model.predict(np.array([[len(priceList)]]))[0]
    else:
        predicted = price

    predictList.append(predicted)

    err = price - predicted
    square_err = err * err
    totalErr.append(square_err)

    if len(totalErr) >= average_size:
        mse = sum(totalErr) / average_size
        rmse = np.sqrt(mse)
        rmseList.append(rmse)
        totalErr.clear()

    df_price = pd.DataFrame(list(zip(dateList, priceList, predictList)), columns=['datetime', 'price', 'predicted'])
    df_rmse = pd.DataFrame(list(zip(dateList[-len(rmseList):], rmseList)), columns=['datetime', 'rmse'])

    price_figure = {
        'data': [
            go.Scatter(x=df_price['datetime'], y=df_price['price'], name='Actual Price', mode='lines+markers'),
            go.Scatter(x=df_price['datetime'], y=df_price['predicted'], name='Predicted Price', mode='lines+markers')
        ]
    }

    rmse_figure = {
        'data': [
            go.Scatter(x=df_rmse['datetime'], y=df_rmse['rmse'], name='RMSE', mode='lines+markers')
        ]
    }

    return price_figure, rmse_figure

app.run_server()


C:\Users\surawich\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  
C:\Users\surawich\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\surawich\Anaconda3\lib\site-packages\pyarrow\compute.py:198: RuntimeWarning: Python binding for CumulativeSumOptions not exposed
  .format(class_name), RuntimeWarning)
C:\Users\surawich\Anaconda3\lib\site-packages\pyarrow\compute.py:198: RuntimeWarning: Python binding for RankOptions not exposed
  .format(class_name), RuntimeWarning)
